In [1]:
import numpy as np

In [12]:
import pandas as pd

In [2]:
from sklearn.preprocessing import OneHotEncoder

In [3]:
X = np.load('../malaria/x_values.npy')

In [4]:
X[0]

array([[237, 237, 237, ..., 128, 129, 133],
       [237, 237, 237, ..., 130, 132, 134],
       [237, 237, 237, ..., 130, 128, 130],
       ...,
       [ 68,  61,  58, ..., 242, 242, 242],
       [ 74,  64,  57, ..., 245, 245, 242],
       [ 79,  66,  59, ..., 245, 245, 245]], dtype=uint8)

In [5]:
X.shape

(1000, 108, 108)

In [6]:
y = np.load('../malaria/y_values.npy')

In [7]:
y.shape

(1000,)

In [8]:
ohe = OneHotEncoder(sparse_output=False)

In [9]:
ohe.fit(y.reshape(-1,1))

OneHotEncoder(sparse_output=False)

In [10]:
encoded_y = ohe.transform(y.reshape(-1,1))

In [11]:
encoded_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [58]:
encoded_y.shape

(1000, 7)

In [14]:
y_df = pd.DataFrame(encoded_y)
y_df

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,1.0,0.0,0.0,0.0
996,0.0,0.0,0.0,1.0,0.0,0.0,0.0
997,0.0,0.0,0.0,1.0,0.0,0.0,0.0
998,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [27]:
column_labels=list(ohe.get_feature_names_out())
column_labels

['x0_difficult',
 'x0_gametocyte',
 'x0_leukocyte',
 'x0_red blood cell',
 'x0_ring',
 'x0_schizont',
 'x0_trophozoite']

In [33]:
y_df.columns=column_labels
y_df

,x0_difficult,x0_gametocyte,x0_leukocyte,x0_red blood cell,x0_ring,x0_schizont,x0_trophozoite
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,1.0,0.0,0.0,0.0
996,0.0,0.0,0.0,1.0,0.0,0.0,0.0
997,0.0,0.0,0.0,1.0,0.0,0.0,0.0
998,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [43]:
# ARCHITECTURE imports

import tensorflow as tf
import numpy as np
from tensorflow.keras import models, Sequential, layers, regularizers

In [44]:
from keras.layers import Dense

In [35]:
X = np.expand_dims(X,axis=3)

In [36]:
X.shape

(1000, 108, 108, 1)

In [52]:
y.shape

(1000,)

In [54]:
y = np.expand_dims(y,axis=1)
y.shape

(1000, 1)

In [38]:
image_shape = (X.shape[1], X.shape[2], X.shape[3])
image_shape

(108, 108, 1)

In [45]:
model = Sequential() 

model.add(layers.Conv2D(filters=128, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(filters=32, kernel_size=(2,2), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))

# Dropouts reduce overfitting by randomly turning neurons off during training.

model.add(layers.Dropout(0.2))

# categorical = softmax
model.add(Dense(7, activation='softmax'))

In [46]:
# COMPILATION imports

from tensorflow.keras import optimizers, callbacks

In [47]:
#### 2. COMPILATION

adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)

In [60]:
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [49]:
# FIT imports

from tensorflow.keras import callbacks

In [50]:
#### 3. FIT

es = callbacks.EarlyStopping(patience=30, restore_best_weights=True)

In [61]:
model.fit(X, encoded_y,
          batch_size=32, # Batch size -too small--> no generalization
          epochs=100,    #            -too large--> slow computations
          validation_split=0.3,
          callbacks=[es],
          verbose=1)

Epoch 1/100
22/22 [==============================] - 14s 601ms/step - loss: 88.9170 - accuracy: 0.7829 - val_loss: 22.1090 - val_accuracy: 0.9700
Epoch 2/100
22/22 [==============================] - 14s 644ms/step - loss: 15.3212 - accuracy: 0.9471 - val_loss: 12.9374 - val_accuracy: 0.9700
Epoch 3/100
22/22 [==============================] - 14s 620ms/step - loss: 12.3137 - accuracy: 0.9629 - val_loss: 11.1592 - val_accuracy: 0.9700
Epoch 4/100
22/22 [==============================] - 14s 648ms/step - loss: 10.5029 - accuracy: 0.9657 - val_loss: 9.7468 - val_accuracy: 0.9700
Epoch 5/100
22/22 [==============================] - 15s 668ms/step - loss: 9.2048 - accuracy: 0.9657 - val_loss: 8.6363 - val_accuracy: 0.9700
Epoch 6/100
22/22 [==============================] - 14s 638ms/step - loss: 8.2118 - accuracy: 0.9657 - val_loss: 7.7381 - val_accuracy: 0.9700
Epoch 7/100
22/22 [==============================] - 14s 621ms/step - loss: 7.4301 - accuracy: 0.9671 - val_loss: 7.1225 - val_ac

22/22 [==============================] - 13s 606ms/step - loss: 1.4255 - accuracy: 0.9957 - val_loss: 1.6221 - val_accuracy: 0.9533
Epoch 58/100
22/22 [==============================] - 13s 589ms/step - loss: 1.4357 - accuracy: 0.9900 - val_loss: 1.6275 - val_accuracy: 0.9367
Epoch 59/100
22/22 [==============================] - 15s 693ms/step - loss: 1.4521 - accuracy: 0.9729 - val_loss: 1.5766 - val_accuracy: 0.9567
Epoch 60/100
22/22 [==============================] - 14s 661ms/step - loss: 1.4579 - accuracy: 0.9843 - val_loss: 1.5635 - val_accuracy: 0.9633
Epoch 61/100
22/22 [==============================] - 15s 669ms/step - loss: 1.4345 - accuracy: 0.9786 - val_loss: 1.5965 - val_accuracy: 0.9500
Epoch 62/100
22/22 [==============================] - 13s 598ms/step - loss: 1.3896 - accuracy: 0.9929 - val_loss: 1.5528 - val_accuracy: 0.9667
Epoch 63/100
22/22 [==============================] - 13s 582ms/step - loss: 1.3569 - accuracy: 0.9886 - val_loss: 1.5173 - val_accuracy: 0.960

In [68]:
model.evaluate(X, encoded_y)

32/32 [==============================] - 2s 57ms/step - loss: 2.0876 - accuracy: 0.9670


[2.08760404586792, 0.9670000076293945]